In [1]:
import glob
import pandas as pd
# import numpy as np

In [37]:
relations = pd.read_csv("mars13-20/relationsWorld2022-03-20.csv", header=None)
relations = relations.rename(columns={0:"Viewers_list", 1:"Streamer", 2:"WatchTime"})

In [38]:
relations

,Viewers_list,Streamer
0,"['0000009mil', '0001jnfps7', '0001ray', '00023...",loud_coringa
1,"['0000_01', '000jose008', '000sunflower000', '...",ElSpreen
2,"['000_fnwortas_000', '00_blast', '00floffi', '...",ROSHTEIN
3,"['000guitarman', '001aroha', '009soundsystemdr...",HasanAbi
4,"['00bidub', '00rioku00', '00ysera00', '0110011...",Maximum
...,...,...
17486,"['00schneida', '00tefl00', '012hero012', '08xe...",TrilluXe
17487,"['0arsoluto', '10obelix100', '11mike33', '1705...",ricardoquintacolumnero
17488,"['112358sum', '16southpaw', '1drr', '1holyreli...",Day9tv
17489,"['000enricpuig26', '0hgodno', '10050816', '14d...",XQN_TheSad


In [39]:
# 1m40 
# all_relations = glob.glob("mars13-20/relations*.csv")
# relations = pd.concat((pd.read_csv(f, header=None) for f in all_relations), axis=0, ignore_index=True)
# relations = relations.rename(columns={0:"Viewers_list", 1:"Streamer", 2:"WatchTime"})
# print(f" ** Memory usage of the file - {sum(relations.memory_usage()) * 0.000001} MB for {len(relations.index)} Rows")

In [40]:
# 2m20s rend la liste "list()"
relations['Viewers_list'] = relations['Viewers_list'].apply(eval)

In [59]:
relations = relations[relations['Viewers_list'].apply(lambda x: len(x) >= 10000)]

In [60]:
relations

,Viewers_list,Streamer
0,"[0000009mil, 0001jnfps7, 0001ray, 00023ppqq, 0...",loud_coringa
1,"[0000_01, 000jose008, 000sunflower000, 0011010...",ElSpreen
2,"[000_fnwortas_000, 00_blast, 00floffi, 00h01, ...",ROSHTEIN
3,"[000guitarman, 001aroha, 009soundsystemdreamsc...",HasanAbi
4,"[00bidub, 00rioku00, 00ysera00, 01100110_, 01c...",Maximum
...,...,...
17352,"[00jabba, 00mkin, 0102rekoj, 0221sense, 040_ti...",NoWay4u_Sir
17353,"[03day, 0_bonkers_0, 0geebee0, 0july0, 0muffin...",daltoosh
17355,"[0000_udinrosli, 02mynameisjeff, 02vos, 036alm...",fuslie
17357,"[00jaredavis, 032bit, 03c63ef3, 04_00, 07jungh...",MOONMOON


In [61]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

In [62]:
# 45s 
viewers_1D = to_1D(relations["Viewers_list"])
viewers_1D_count = viewers_1D.value_counts()

In [63]:
# WARNING DO NOT EXECUTE TWICE
# viewers_1D.to_csv("mars13-20/viewers1D.csv", mode='a')

Les valeurs à suivre sont en quarts d'heure.

In [64]:
viewers_1D_count.head(3) # les bots :)

commanderroot     1818
spiketrapclair    1682
aten              1594
dtype: int64

In [65]:
viewers_1D_count.describe()

count    4.855261e+06
mean     7.669991e+00
std      1.083622e+01
min      1.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      1.000000e+01
max      1.818000e+03
dtype: float64

In [66]:
# Quarts_heure = (np.datetime64("2022-02-14 00:00")-np.datetime64("2022-02-13 00:00")).astype(int)/15
# Quarts_heure
Quarts_heure = 96

In [67]:
def temps(x, streamersRegardés=2):
    print(f"{x/4:.2f} heures regardées, soit {x/4/24:.2f} jours sur un total de {Quarts_heure/4/24:.2f} jours étudiés.")
    print(f"Ou encore {streamersRegardés} streamers regardés en parallèle pendant {(x/streamersRegardés)/(Quarts_heure/24):.2f} heures par jour.")

In [68]:
temps(viewers_1D_count.quantile(0.9999), 4)

45.75 heures regardées, soit 1.91 jours sur un total de 1.00 jours étudiés.
Ou encore 4 streamers regardés en parallèle pendant 11.44 heures par jour.


1 personne sur 10000 est soit un bot, soit un dégénéré qui mutliplexe 2 streams en même temps pendant 19h par jour.

In [69]:
def RejectedPseudo(minHeure=1):
    temp = list(viewers_1D_count[viewers_1D_count>viewers_1D_count.quantile(0.9999)].index) + list(viewers_1D_count[viewers_1D_count<4*minHeure].index)
    print(f"{len(temp)} viewers rejetés sur {len(viewers_1D_count)}")
    return temp

In [70]:
exiled = RejectedPseudo()

2255254 viewers rejetés sur 4855261


In [71]:
# 35s c'est insane pandas ptdr
gephi = relations[['Viewers_list', 'Streamer']].explode('Viewers_list')
gephi = gephi.rename(columns={'Viewers_list': 'source', 'Streamer': 'target'})

In [72]:
# 30s
gephi = gephi[~gephi['source'].isin(exiled)]

In [ ]:

streamerviewer = gephi['target'].unique()
gephi = gephi[~gephi['source'].isin(streamerviewer)]

In [73]:
gephi['temp']=1

In [74]:
# 1m20s
gephi = gephi.groupby(by=['source', 'target']).agg({'temp':'count'})
gephi = gephi.reset_index().rename(columns={'temp': 'weight'})

In [75]:
gephi.head()

,source,target,weight
0,000000000000,Distortion2,1
1,000000000000,Jerma985,7
2,000000000000,forsen,1
3,00000000000000,xQcOW,8
4,0000000000000000,Ninja,4


In [76]:
gephi.shape

(4075553, 3)

In [25]:
# 30s
gephi.to_csv('twitchGephi.csv', mode='w', encoding='utf-8')